In [1]:
%load_ext autoreload
%autoreload 2

### imports and utilities

In [2]:
!pip install nltk scikit-learn matplotlib scipy -q

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>


False

In [4]:
from collections import Counter
from utils import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import joblib
from tqdm.notebook import tqdm
from scipy.special import softmax
from scipy.stats import norm
from scipy.stats import entropy as calculate_entropy

from sklearn.naive_bayes import GaussianNB, MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### load dataset

In [5]:
dataset = "newsgroup"

# total number of samples needed
randomize = False

# retrieve dataset
categories = ['rec.autos', 'talk.politics.mideast', 'alt.atheism', 'sci.space']

all_docs = fetch_20newsgroups(subset='train', shuffle=randomize, remove=('headers', 'footers', 'quotes'), categories=categories)
all_docs, old_labels, categories = all_docs.data, all_docs.target, all_docs.target_names

In [6]:
# dataset = "bbc"

# data = pd.read_csv('bbcsport.csv')

# all_docs = data["text"].to_list()
# old_labels = data["topic"].to_list()
# categories = classes = np.unique(data["topic"]).tolist()

### clean dataset

In [7]:
datasize = 40
min_document_length = 160
max_document_length = 256


index = -1
docs, labels, label_indices = [], [], []

sizes = [0]*len(categories)

with tqdm(total=len(categories)*datasize) as pbar:
    while sum(sizes) < len(categories)*datasize:
        index += 1
        label_index = old_labels[index]
            
        if sizes[label_index] == datasize:
            continue
        
        doc = all_docs[index]
        status, doc, word_count = clean_doc(doc, True)
        
        if not status:
            continue
            
        if min_document_length is not None and len(doc) < min_document_length:
            continue
            
        if max_document_length is not None and len(doc) > max_document_length:
            continue
        
        label_indices.append(label_index)
        labels.append(categories[label_index])
        
        docs.append(doc)
        sizes[label_index] += 1
        pbar.update(1)

labels = np.array(labels)
label_indices = np.array(label_indices)

In [8]:
doc_index = 3
print(f"Topic: {labels[doc_index]}\n{'='*50}\n{docs[doc_index][:512]}")

Topic: rec.autos
not to mention my friend s 54 citroen traction avant with the light switch and dimmer integrate in a single stalk off the steer column those dumb french be apparently copying the japanese before the german


In [9]:
print(sizes)
assert min(sizes) == max(sizes) == datasize

[40, 40, 40, 40]


### Split data

In [10]:
x_train, x_test, y_train, y_test = train_test_split(docs, labels, test_size =.3)

In [11]:
print(f"there are {len(docs)} total docs, {len(y_train)} train and {len(y_test)} test")

there are 160 total docs, 112 train and 48 test


### Initialize Vectorizer

In [12]:
vectorizer_type = "not-tfidf"

# initialize the count vectorizer
if vectorizer_type == "tfidf":
    vectorizer = TfidfVectorizer
else:
    vectorizer = CountVectorizer()

# fit it to dataset
vectorizer.fit(x_train)

vocabulary = np.array(vectorizer.get_feature_names())
print("word_count is", len(vocabulary))

word_count is 1474


### Prepare Datatset

In [13]:
# create doc count vectors
train_doc_vectors = vectorizer.transform(x_train).toarray()
test_doc_vectors = vectorizer.transform(x_test).toarray()

wdf_train = pd.DataFrame(train_doc_vectors, columns=vocabulary)
wdf_test = pd.DataFrame(test_doc_vectors, columns=vocabulary)

### some new method

In [14]:
wdf_train.head()

,000,10,1000,12,16th,1940,1982,1984,1987,1990,...,yahud,yeah,year,yes,yo,yorker,you,your,yourself,zuma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
# joblib.load("word_word_co_nb.joblib")

In [ ]:
word_word_co_nb = {}

for word in tqdm(vocabulary):
    word_word_co_nb[word] = {}
    
    for given_word in (vocabulary):
        word_word_co_nb[word][given_word] = MultinomialNB()

        X = wdf_train[given_word].to_numpy(dtype=float).reshape((-1, 1))
        y = (wdf_train[word] > 0).astype(float)
        word_word_co_nb[word][given_word].fit(X, y)

In [ ]:
# joblib.dump(word_word_co_nb, "word_word_co_nb.joblib")

In [18]:
alpha = 0
wdf_train_prime = wdf_train.copy()

wdt_train = wdf_train_prime.copy()
wdt_train["__labels__"] = y_train

word_doc_count = wdf_train_prime.sum(0)
word_word_pr_distr = pd.DataFrame(data=0.0, columns=vocabulary, index=vocabulary)

for word in tqdm(vocabulary):
    pxy = (wdf_train_prime[wdf_train_prime[word] > 0].sum(0) + alpha) / (word_doc_count[word] + alpha)
    word_word_pr_distr[word] = pxy * (word_doc_count[word] / word_doc_count)

print(f"word_word_pr_distr shape = {word_word_pr_distr.shape}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1443/1443 [00:07<00:00, 187.04it/s]

word_word_pr_distr shape = (1443, 1443)


In [28]:
word = "space"

In [29]:
wdf_train[[word]]#[wdf_train[word] > 0]

,space
0,0
1,0
2,0
3,0
4,0
...,...
107,0
108,0
109,0
110,0


In [530]:
word_word_pr_distr.head()

,000,031349,10,1000,11,14,16th,1900,1940,1982,...,yahud,yeah,year,yes,yo,yorker,you,your,yourself,zeuge
000,1.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0
031349,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
10,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [534]:
entropy_values = calculate_entropy(word_word_pr_distr, axis=0) * calculate_entropy(word_word_pr_distr, axis=1)
# entropy_values_norm = entropy_values / entropy_values.max()
entropy_values_norm = (entropy_values.max() - entropy_values) / entropy_values.max()
entropy_values_norm = pd.DataFrame(data=entropy_values_norm, index=word_word_pr_distr.columns)[0]
entropy_values_norm.head()

000       0.658496
031349    0.762590
10        0.773685
1000      0.739970
11        0.688549
Name: 0, dtype: float64

In [535]:
entropy_values_norm.loc["the"], entropy_values_norm.loc["east"]

(0.0, 0.6279044392802534)

In [536]:
y_train[:10]

array(['rec.autos', 'talk.politics.mideast', 'talk.politics.mideast',
       'rec.autos', 'rec.autos', 'sci.space', 'alt.atheism', 'rec.autos',
       'alt.atheism', 'sci.space'], dtype='<U21')

In [686]:
# doc_indices = np.arange(len(y_train))
# np.random.shuffle(doc_indices)

# doc_indices = doc_indices[:30]
doc_indices = [0, 3, 1, 2, 5, 9, 6, 8]
for doc_index in doc_indices[:3]:
    print(f"Topic: {y_train[doc_index]}\n{'='*50}\n{x_train[doc_index][:512]}")
    print()

Topic: rec.autos
not to mention my friend s 54 citroen traction avant with the light switch and dimmer integrate in a single stalk off the steer column those dumb french be apparently copying the japanese before the german

Topic: rec.autos
right in the thirty both buick and packard have two spare mount in well in the front fender of course that be back when the front fender be long enough to provide room there be a couple of other marque that do this a well but memory fade

Topic: talk.politics.mideast
to my fellow columbian i must ask why do you say that i engage in fantasy arafat be a terrorist who happens to have a lot of pull among palestinian can we ignore the two fact i doubt it



In [687]:
doc_words_li = []

counter = 0
for index, doc_index in enumerate(range(len(y_train))):
    doc_words_li.append(wdf_train.iloc[doc_index][wdf_train.iloc[doc_index] > 0].index.tolist())
    
    if counter < 3 and doc_index in doc_indices:
        counter += 1
        print(doc_words_li[-1], "\n")

['54', 'and', 'apparently', 'avant', 'be', 'before', 'citroen', 'column', 'copying', 'dimmer', 'dumb', 'french', 'friend', 'german', 'in', 'integrate', 'japanese', 'light', 'mention', 'my', 'not', 'off', 'single', 'stalk', 'steer', 'switch', 'the', 'those', 'to', 'traction', 'with'] 

['among', 'arafat', 'ask', 'be', 'can', 'columbian', 'do', 'doubt', 'engage', 'fact', 'fantasy', 'fellow', 'happens', 'have', 'ignore', 'in', 'it', 'lot', 'must', 'my', 'of', 'palestinian', 'pull', 'say', 'terrorist', 'that', 'the', 'to', 'two', 'we', 'who', 'why', 'you'] 

['above', 'anyone', 'be', 'conception', 'concern', 'concrete', 'creation', 'detail', 'differ', 'do', 'during', 'fit', 'force', 'have', 'how', 'idea', 'it', 'know', 'list', 'next', 'of', 'palestinian', 'police', 'proposal', 'read', 'some', 'stage', 'talk', 'that', 'the', 'there', 'this', 'will', 'with'] 



In [688]:
dfx_li = []

for doc_index1 in tqdm(range(len(y_train))):
    dfx_li.append([])
    
    if doc_index1 not in doc_indices:
        continue
        
    for doc_index2 in range(len(y_train)):
        dfx = pd.DataFrame(data=0.0, index=doc_words_li[doc_index1], columns=doc_words_li[doc_index2])

        for word in doc_words_li[doc_index1]:
            for other_word in doc_words_li[doc_index2]:
                dfx[other_word][word] =  word_word_pr_distr[other_word][word]
        
        dfx_li[-1].append(dfx)
dfx.head()

,and,bastard,be,definitely,for,ghod,have,just,nls,of,...,sub,suitable,thank,that,the,thought,use,version,will,would
be,0.443243,0.021622,1.0000,0.021622,0.210811,0.021622,0.340541,0.124324,0.021622,0.545946,...,0.021622,0.021622,0.021622,0.491892,0.713514,0.043243,0.07027,0.021622,0.151351,0.32973
engineering,0.000000,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.00000,0.000000,0.000000,0.50000
get,0.375000,0.000000,0.8125,0.000000,0.375000,0.000000,0.125000,0.125000,0.000000,0.687500,...,0.000000,0.000000,0.000000,0.562500,0.687500,0.000000,0.12500,0.000000,0.250000,0.18750
glad,0.000000,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.00000
griffin,0.000000,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.00000


In [689]:
latent_vector = []
counter = 0

for doc_index1 in range(len(y_train)):
    latent_vector.append([])
    
    if doc_index1 not in doc_indices:
        continue
        
    values = []
    
    for doc_index2 in range(len(y_train)):
        score = dfx_li[doc_index1][doc_index2].max(0).mean()
        
        latent_vector[-1].append(score)
        values.append([y_train[doc_index2], score])
    
    
    if counter < 10:
        counter += 1
        print(y_train[doc_index1])
        print(pd.DataFrame(values, columns=["topic", "score"]).sort_values(ascending=False, by="score").head(10))
        print()
    
latent_vector = pd.DataFrame(data=latent_vector, index=wdf_train.index, columns=wdf_train.index).fillna(0)

rec.autos
                    topic     score
0               rec.autos  1.000000
27              rec.autos  0.626777
65            alt.atheism  0.614294
94              rec.autos  0.604420
98            alt.atheism  0.591789
29              sci.space  0.581703
44  talk.politics.mideast  0.566667
52  talk.politics.mideast  0.505164
22              rec.autos  0.489391
35              rec.autos  0.467949

talk.politics.mideast
                     topic     score
1    talk.politics.mideast  1.000000
102  talk.politics.mideast  0.693548
106  talk.politics.mideast  0.660256
2    talk.politics.mideast  0.627451
105            alt.atheism  0.614286
11               rec.autos  0.611765
53             alt.atheism  0.586420
70               rec.autos  0.580645
84             alt.atheism  0.563333
3                rec.autos  0.556190

talk.politics.mideast
                     topic     score
2    talk.politics.mideast  1.000000
1    talk.politics.mideast  0.639394
41               rec.autos  0.

In [690]:
latent_vector.head()

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,1.000000,0.368306,0.274299,0.359069,0.226014,0.231615,0.266385,0.329110,0.352957,0.307603,...,0.386643,0.319239,0.269104,0.277672,0.343275,0.359354,0.248500,0.245263,0.355845,0.301125
1,0.296448,1.000000,0.627451,0.556190,0.236089,0.280072,0.316499,0.418141,0.447408,0.340092,...,0.693548,0.367808,0.441667,0.614286,0.660256,0.444680,0.350826,0.415857,0.464619,0.333368
2,0.268172,0.639394,1.000000,0.415045,0.245254,0.281533,0.254417,0.438095,0.438683,0.310165,...,0.443944,0.481250,0.372959,0.352914,0.425705,0.456583,0.359111,0.309526,0.524117,0.360994
3,0.253981,0.563805,0.407353,1.000000,0.268570,0.278730,0.306323,0.422063,0.435156,0.344533,...,0.444087,0.356200,0.373848,0.325352,0.422285,0.416794,0.360159,0.304796,0.505376,0.344444
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [691]:
n_clusters = 4

# cluster the topics
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(latent_vector.loc[doc_indices])
print("latent_vector, kmeans_inertia = ", kmeans.inertia_)

for ii in range(n_clusters):
    print(Counter(y_train[doc_indices][kmeans.labels_ == ii]))

print()

latent_vector, kmeans_inertia =  3.6781928368289742
Counter({'talk.politics.mideast': 2, 'rec.autos': 1})
Counter({'sci.space': 1})
Counter({'rec.autos': 1})
Counter({'alt.atheism': 2, 'sci.space': 1})



In [692]:
n_clusters = 4

wdf_train_prime = wdf_train.copy()
wdf_test_prime = wdf_test.copy()
    
# cluster the topics
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(wdf_train_prime)
print("wdf_train, kmeans_inertia = ", kmeans.inertia_)

clusters = []
categories_prime = list(range(n_clusters))

y_train_prime = kmeans.predict(wdf_train_prime)
y_test_prime = kmeans.predict(wdf_test_prime)

for ii in range(n_clusters):
    clusters.append(Counter(y_train[y_train_prime == ii]))
    print(clusters[-1])

print()

wdf_train, kmeans_inertia =  4230.109369977496
Counter({'sci.space': 17, 'talk.politics.mideast': 16, 'rec.autos': 16, 'alt.atheism': 12})
Counter({'talk.politics.mideast': 3, 'alt.atheism': 2, 'rec.autos': 1})
Counter({'sci.space': 10, 'rec.autos': 8, 'talk.politics.mideast': 6, 'alt.atheism': 4})
Counter({'alt.atheism': 7, 'talk.politics.mideast': 4, 'sci.space': 4, 'rec.autos': 2})



In [693]:
# y_train[doc_indices]

In [694]:
doc_index1 = 0
print(f"Topic: {y_train[doc_index1]}\n{'='*50}\n{x_train[doc_index1][:512]}")
print()

doc_index2 = 3
print(f"Topic: {y_train[doc_index2]}\n{'='*50}\n{x_train[doc_index2][:512]}")

Topic: rec.autos
not to mention my friend s 54 citroen traction avant with the light switch and dimmer integrate in a single stalk off the steer column those dumb french be apparently copying the japanese before the german

Topic: rec.autos
right in the thirty both buick and packard have two spare mount in well in the front fender of course that be back when the front fender be long enough to provide room there be a couple of other marque that do this a well but memory fade


In [695]:
dfx_li[doc_index1][doc_index2].max(1).sort_values(ascending=False).head(10)

integrate     1.0
in            1.0
and           1.0
apparently    1.0
avant         1.0
be            1.0
before        1.0
citroen       1.0
column        1.0
copying       1.0
dtype: float64

In [696]:
dfx_li[doc_index1][doc_index2].mean(1).sort_values(ascending=False).head(10)

in         0.216949
the        0.187317
be         0.187181
light      0.185714
to         0.184783
and        0.181193
mention    0.171429
steer      0.171429
single     0.171429
off        0.171429
dtype: float64

In [697]:
print(xv[xv < 0].mean(0).mean())
xv[xv < 0].mean(0).sort_values(ascending=False).head(10)

-0.5468186228727564


reuseable     -0.328365
reminder      -0.328365
version       -0.328365
nls           -0.328365
bastard       -0.328365
definitely    -0.328365
thank         -0.328365
suitable      -0.328365
sub           -0.328365
spacelifter   -0.328365
dtype: float64

In [549]:
dfx_li[doc_index1][doc_index2].max(0).sort_values(ascending=False).head(16)

that         1.0
be           1.0
have         1.0
the          1.0
police       1.0
will         1.0
with         1.0
you          0.6
believe      0.5
close        0.5
detection    0.5
detector     0.5
off          0.5
enough       0.5
forget       0.5
frequency    0.5
dtype: float64

In [550]:
dfx_li[doc_index1][doc_index2].max(1).sort_values(ascending=False).head(16)

read          1.0
with          1.0
will          1.0
be            1.0
conception    1.0
concern       1.0
concrete      1.0
creation      1.0
detail        1.0
differ        1.0
during        1.0
fit           1.0
force         1.0
have          1.0
idea          1.0
above         1.0
dtype: float64

In [210]:
wdf_train.iloc[idx2].sort_values(ascending=True)

000         0
pity        0
pimentel    0
pilot       0
piece       0
           ..
to          2
fuel        2
how         2
the         2
be          3
Name: 7, Length: 1467, dtype: int64

In [186]:
dfx.max(0).sort_values(ascending=True).head(15)

memory     0.050847
thirty     0.050847
buick      0.050847
room       0.050847
packard    0.050847
fade       0.050847
fender     0.050847
marque     0.050847
spare      0.067797
course     0.067797
mount      0.067797
front      0.067797
two        0.080000
both       0.107317
when       0.120000
dtype: float64

In [183]:
dfx.sum(0).sort_values(ascending=False).head(15)

the       4.303757
be        3.994595
to        3.977974
and       3.803540
in        3.734835
of        3.445151
that      2.753275
have      2.407538
do        2.102149
this      2.010799
but       1.688663
there     1.236046
well      0.891024
couple    0.836783
long      0.835313
dtype: float64

In [129]:
result = []
doc_index = 3
print(f"Topic: {labels[doc_index]}\n{'='*50}\n{docs[doc_index][:512]}")

width = 0
for i, word in enumerate(wdf_train.columns):
    result.append(gnbs[word].predict_log_proba(wdf_test.loc[[doc_index]])[0])
    if len(result) > width:
        width = len(result)

res = pd.DataFrame(data=result, index=wdf_train.columns).fillna(0)

Topic: rec.autos
not to mention my friend s 54 citroen traction avant with the light switch and dimmer integrate in a single stalk off the steer column those dumb french be apparently copying the japanese before the german


In [134]:
res.sum(1).sort_values()

and     -5.121664e+10
the     -5.065995e+10
mount   -3.989702e+10
spare   -3.989702e+10
issue   -3.711350e+10
             ...     
make    -1.855667e+09
we      -1.855666e+09
at      -1.670100e+09
me      -1.670099e+09
get     -1.113398e+09
Length: 1467, dtype: float64

In [113]:
res[1].sort_values(ascending=True).head(20)

be          -4.082481e+09
the         -7.422644e+08
that        -5.567038e+08
you         -3.711371e+08
of          -1.855654e+08
not         -4.879017e+03
play         0.000000e+00
planetary    0.000000e+00
plane        0.000000e+00
placement    0.000000e+00
place        0.000000e+00
pity         0.000000e+00
pimentel     0.000000e+00
pilot        0.000000e+00
piece        0.000000e+00
pickup       0.000000e+00
phone        0.000000e+00
phobos       0.000000e+00
peter        0.000000e+00
pertains     0.000000e+00
Name: 0, dtype: float64

In [99]:
np.log(0)

C:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


-inf

In [27]:
wdf_train_prime.head()

,000,031349,10,1000,11,14,16th,1900,1940,1982,...,yeah,year,yes,yo,yorker,you,your,yourself,zeuge,__labels__
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [64]:
kmeans.predict(wdf_train_prime)

array([2, 0, 2, 2, 2, 0, 0, 3, 1, 2, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 3, 0,
       0, 0, 3, 3, 2, 0, 2, 2, 2, 3, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 1, 0, 3, 0, 3, 3, 2, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 0, 0, 3, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 1, 2, 0, 0, 3, 3, 0, 0,
       0, 0, 3, 2, 3, 0, 0, 1, 0, 2, 0, 2, 1, 0, 1, 0, 0, 0, 0, 3, 0, 2,
       0, 3])

In [18]:
# the word topic distr
wdt_train = wdf_train_prime.copy()
wdt_test = wdf_test_prime.copy()

wdt_test["__labels__"] = y_test_prime
wdt_train["__labels__"] = y_train_prime

word_doc_count = wdf_train_prime.sum(0)
topic_doc_count = pd.DataFrame(Counter(y_train_prime), index=[0]).T[0]
word_topic_pr_distr = pd.DataFrame(data=0.0, columns=categories_prime, index=vocabulary)

for category in tqdm(categories_prime):
    pxy = wdf_train_prime[wdt_train["__labels__"] == category].sum(0) / topic_doc_count[category]
    word_topic_pr_distr[category] = pxy * (topic_doc_count[category] / word_doc_count)

print(f"word_topic_pr_distr shape = {word_topic_pr_distr.shape}")


word_topic_pr_distr shape = (1467, 4)


In [19]:
word_topic_pr_distr.head()

,0,1,2,3
000,1.0,0.0,0.0,0.0
031349,1.0,0.0,0.0,0.0
10,1.0,0.0,0.0,0.0
1000,1.0,0.0,0.0,0.0
11,0.0,0.0,1.0,0.0


In [62]:
word_topic_entropy = calculate_entropy(word_topic_pr_distr, axis=1)
word_topic_entropy_norm = (word_topic_entropy.max() - word_topic_entropy) / word_topic_entropy.max()

order_factor = word_topic_entropy_norm #* (word_doc_count / word_doc_count.max())

order_indices = order_factor.argsort()[::-1]
word_topic_pr_distr.iloc[order_indices].head(10)

,0,1,2,3
zeuge,1.0,0.0,0.0,0.0
ftp,0.0,0.0,1.0,0.0
full,0.0,1.0,0.0,0.0
fundraise,0.0,0.0,1.0,0.0
funny,1.0,0.0,0.0,0.0
further,1.0,0.0,0.0,0.0
fusi,1.0,0.0,0.0,0.0
gain,1.0,0.0,0.0,0.0
gallon,0.0,0.0,0.0,1.0
game,0.0,1.0,0.0,0.0


In [61]:
word = "space"
calculate_entropy(word_topic_pr_distr.loc[word]), npword_doc_count[word]

(0.6730116670092565, 5)

In [60]:
word_doc_count.mean

<bound method Series.mean of 000          2
031349       1
10           1
1000         1
11           2
            ..
yorker       1
you         87
your        23
yourself     1
zeuge        1
Length: 1467, dtype: int64>

In [20]:
# the entropy and softma for reducing relation
word_topic_pr_entropy = calculate_entropy(word_topic_pr_distr, axis=1)

# the norm freq of the word in topic
word_topic_pr_freq_weight = word_topic_pr_distr.sum(1)


# word_topic_pr_infl = 1 - softmax(word_topic_pr_entropy)

# influence
# wdf_train_prime *= word_topic_pr_infl

In [21]:
wdf_train_prime.max().max()

7

In [22]:
word_topic_pr_freq_weight.max()

1.0

In [23]:
word_topic_pr_entropy

array([0.       , 0.       , 0.       , ..., 1.0554635, 0.       ,
       0.       ])